In [44]:
import pandas as pd
import numpy as np
import seaborn as sns
from pathlib import Path as P
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['KaiTi', 'SimHei', 'FangSong']  # 汉字字体,优先使用楷体，如果找不到楷体，则使用黑体
plt.rcParams['font.size'] = 12  # 字体大小
plt.rcParams['axes.unicode_minus'] = False  # 正常显示负号
from collections import defaultdict
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler

这里是要把问题所需的训练数据整理出来，基本上就是一个机器学习的数据集获取的过程。

这是土壤湿度W，每月的数据。注意：代码里面用符号H表示，实际上是W。

In [32]:
df = pd.read_csv('./附件3-湿度-预处理.csv', index_col='time')
df_H=df.drop(columns=['month','year'])
df_H.columns

Index(['10cm湿度(kg/m2)', '40cm湿度(kg/m2)', '100cm湿度(kg/m2)', '200cm湿度(kg/m2)'], dtype='object')

这是土壤蒸发量E，每月的数据，和湿度W是对齐的（数据的时间范围完全相同），注意：代码里面用符号SoilEvap表示，实际上是E。

In [33]:
df = pd.read_csv('./附件4-土壤蒸发量-预处理.csv', index_col='time')
df_SoilEvap=df.drop(columns=['month','year'])
df_SoilEvap.columns

Index(['土壤蒸发量(W/m2)', '土壤蒸发量(mm)'], dtype='object')

这是气候条件，有很多变量，但大部分变量和要预测的湿度相关性不大（见相关性分析）。
筛选了4个相关性比较大的变量：
```
['土壤蒸发量(mm)', '平均露点温度(℃)', '最低气温极值(℃)', '平均最低气温(℃)']
```

In [34]:
df = pd.read_csv('附件8-气候-预处理.csv',index_col='time')
df_Climate=df.drop(columns=['month','year'])

In [35]:
df_All = df_Climate.join([df_H, df_SoilEvap])

In [36]:
df_All.columns

Index(['平均气温(℃)', '平均最高气温(℃)', '平均最低气温(℃)', '最高气温极值(℃)', '最低气温极值(℃)',
       '平均气温≥18℃的天数', '平均气温≤0℃的天数', '平均露点温度(℃)', '降水量(mm)', '最大单日降水量(mm)',
       '降水天数', '平均海平面气压(hPa)', '最低海平面气压(hPa)', '平均站点气压(hPa)', '平均能见度(km)',
       '最小能见度(km)', '最大能见度(km)', '平均风速(knots)', '平均最大持续风速(knots)',
       '单日最大平均风速(knots)', '10cm湿度(kg/m2)', '40cm湿度(kg/m2)', '100cm湿度(kg/m2)',
       '200cm湿度(kg/m2)', '土壤蒸发量(W/m2)', '土壤蒸发量(mm)'],
      dtype='object')

In [37]:
df_All.index

Index(['2012-01-01', '2012-02-01', '2012-03-01', '2012-04-01', '2012-05-01',
       '2012-06-01', '2012-07-01', '2012-08-01', '2012-09-01', '2012-10-01',
       ...
       '2021-10-01', '2021-11-01', '2021-12-01', '2022-01-01', '2022-02-01',
       '2022-03-01', '2022-04-01', '2022-05-01', '2022-06-01', '2022-07-01'],
      dtype='object', name='time', length=127)

In [38]:
# 因变量：200 的曲线基本没变化，随机数就行。
Y_COLS=['10cm湿度(kg/m2)', '40cm湿度(kg/m2)', '100cm湿度(kg/m2)',]
# 自变量：蒸发量有两个单位，取其中一个就行，两个是高度相关的，就是等价的。
X_COLS=['土壤蒸发量(mm)', '降水量(mm)', '平均气温(℃)']

这里定义了3个因变量和5个自变量。其中200cm深度土壤湿度因为变化幅度很小（在个位及以下变化），
不需要预测，可以看做常量。

In [39]:
df_train=df_All[Y_COLS+X_COLS]
df_train.columns

Index(['10cm湿度(kg/m2)', '40cm湿度(kg/m2)', '100cm湿度(kg/m2)', '土壤蒸发量(mm)',
       '降水量(mm)', '平均气温(℃)'],
      dtype='object')

这里获取了模型预测所需要的数据，即3个自变量和3个因变量。

In [40]:
df_train.to_csv('问题2-原始训练数据.csv')

In [41]:
df_train

,10cm湿度(kg/m2),40cm湿度(kg/m2),100cm湿度(kg/m2),土壤蒸发量(mm),降水量(mm),平均气温(℃)
time,,,,,,
2012-01-01,13.73,30.87,42.79,0.61,2.29,-22.94
2012-02-01,13.00,30.87,42.79,0.86,2.29,-17.86
2012-03-01,12.60,30.87,42.79,1.91,21.84,-7.11
2012-04-01,11.97,30.73,42.76,7.20,6.10,6.94
2012-05-01,14.18,29.99,42.58,18.10,50.80,14.61
...,...,...,...,...,...,...
2022-03-01,14.96,52.13,93.44,5.98,115.57,-2.36
2022-04-01,NaN,NaN,NaN,NaN,3.30,7.61
2022-05-01,NaN,NaN,NaN,NaN,13.21,12.95


In [42]:
px.line(df_train).update_layout(xaxis_title='时间',yaxis_title='取值',legend_title='变量', title='问题2 自变量和因变量的时序图')

In [43]:
X_train=df_train[X_COLS]
Y_train=df_train[Y_COLS]
X_train, Y_train

(            土壤蒸发量(mm)  降水量(mm)  平均气温(℃)
 time                                   
 2012-01-01       0.61     2.29   -22.94
 2012-02-01       0.86     2.29   -17.86
 2012-03-01       1.91    21.84    -7.11
 2012-04-01       7.20     6.10     6.94
 2012-05-01      18.10    50.80    14.61
 ...               ...      ...      ...
 2022-03-01       5.98   115.57    -2.36
 2022-04-01        NaN     3.30     7.61
 2022-05-01        NaN    13.21    12.95
 2022-06-01        NaN    27.43    19.59
 2022-07-01        NaN    29.21    23.31
 
 [127 rows x 3 columns],
             10cm湿度(kg/m2)  40cm湿度(kg/m2)  100cm湿度(kg/m2)
 time                                                    
 2012-01-01          13.73          30.87           42.79
 2012-02-01          13.00          30.87           42.79
 2012-03-01          12.60          30.87           42.79
 2012-04-01          11.97          30.73           42.76
 2012-05-01          14.18          29.99           42.58
 ...                   ...        